In [8]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
import sys
path="data/"
inputname="human.fasta"
outputname="human_EFIEP"
name=outputname

In [9]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(path)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r','').replace('\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [33]:
def fetch_singleline_features(sequence):
    alphabet=['A','U','G','C']
    four_nucleotide_count=np.zeros(4)
    nucleotide_vectors=[]
    for index,data in enumerate(sequence):
        four_nucleotide_count[alphabet.index(data)]+=1
        position=index+1
        temp_value=four_nucleotide_count/(position*1.0)
        nucleotide_vectors.extend(temp_value)
    alphabet=['A','U','G','C']
    four_nucleotide_count=np.zeros(4)
    sequence2=list(sequence)[::-1]
    for index,data in enumerate(sequence2):
        four_nucleotide_count[alphabet.index(data)]+=1
        position=index+1
        temp_value=four_nucleotide_count/(position*1.0)
        nucleotide_vectors.extend(temp_value)
    return nucleotide_vectors

In [34]:
sequences=read_fasta_file(path+inputname)
print "sequences-length:{}".format(len(sequences))
print "single-line-sequence-length:{}".format(len(sequences[0]))
features_data=[]
for index,sequence in enumerate(sequences):
    features_data.append(fetch_singleline_features(sequence))
print "features shape :{}".format(np.array(features_data).shape)
pd.DataFrame(features_data).to_csv(path+outputname+".csv",header=None,index=None)

sequences-length:2260
single-line-sequence-length:41
features shape :(2260, 328)


In [35]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [36]:
"""
    cross validation
"""

datapath =path+name+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
print clf.best_score_
gamma=clf.best_params_['gamma']
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12).mean()
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+name+'_predict.csv',header=None,index=False)
ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+name+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

0.802654867257
[[['svmC:32.0gamma:0.03125', 0.80265486725663715, 0.867741935483871, 0.7141592920353982, 0.7141592920353982, 0.8911504424778761, 0.797761473810958, 0.78349514563106804, 0.78349514563106804, 0.61501938825232749, '0.8027', 807.0, 323.0, 123.0, 1007.0, 1130.0, 1130.0]]]
328


True

In [37]:
"""
    cross validation
"""
classifier="SVM"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
gamma=clf.best_params_['gamma']
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['SVMC:32.0gamma:0.03125', 0.80265486725663715, 0.867741935483871, 0.7141592920353982, 0.7141592920353982, 0.8911504424778761, 0.797761473810958, 0.78349514563106804, 0.78349514563106804, 0.61501938825232749, 0.80265486725663704, 807.0, 323.0, 123.0, 1007.0, 1130.0, 1130.0]]]
328


True

In [38]:
"""
    cross validation GBDT
"""
classifier="GBDT"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
GBDT = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
clf = GridSearchCV(GBDT, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
min_samples_split=clf.best_params_['min_samples_split']
min_samples_leaf=clf.best_params_['min_samples_leaf']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(GradientBoostingClassifier(n_estimators=n_estimators,learning_rate=0.1,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(GradientBoostingClassifier(n_estimators=n_estimators,learning_rate=0.1,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['GBDTn_estimators:118max_depth:11min_samples_split:300min_samples_leaf:60', 0.695575221238938, 0.7112810707456979, 0.6584070796460177, 0.6584070796460177, 0.7327433628318584, 0.6945814694131468, 0.68382352941176483, 0.68382352941176483, 0.39223566796638815, 0.69557522123893811, 744.0, 386.0, 302.0, 828.0, 1130.0, 1130.0]]]
328


True

In [39]:
"""
    cross validation RF
"""
classifier="RF"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
RF = RandomForestClassifier(min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
clf = GridSearchCV(RF, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
min_samples_split=clf.best_params_['min_samples_split']
min_samples_leaf=clf.best_params_['min_samples_leaf']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(RandomForestClassifier(n_estimators=n_estimators,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(RandomForestClassifier(n_estimators=n_estimators,min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt'),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['RFn_estimators:10max_depth:5min_samples_split:100min_samples_leaf:70', 0.59601769911504421, 0.6079601990049751, 0.54070796460177, 0.54070796460177, 0.6513274336283186, 0.5934458112805773, 0.57236533957845437, 0.57236533957845437, 0.19322122896111185, 0.59601769911504432, 611.0, 519.0, 394.0, 736.0, 1130.0, 1130.0]]]
328


True

In [40]:
"""
    cross validation NB
"""
classifier="NB"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(GaussianNB(),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(GaussianNB(),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier,ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['NB', 0.57433628318584073, 0.5846774193548387, 0.5132743362831859, 0.5132743362831859, 0.6353982300884956, 0.5710810842815437, 0.54665409990574931, 0.54665409990574931, 0.14979379333953974, 0.57433628318584062, 580.0, 550.0, 412.0, 718.0, 1130.0, 1130.0]]]
328


True

In [41]:
"""
    cross validation
"""
classifier="LR"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
logisReg = LogisticRegression()
C_range = 10.0 ** np.arange(-4,3,1)
grid_parame = dict(C=C_range)
clf = GridSearchCV(logisReg, grid_parame, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_

y_predict=cross_val_predict(LogisticRegression(C=C),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(LogisticRegression(C=C),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"C:"+str(C),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['LRC:100.0', 0.90132743362831858, 0.983991462113127, 0.815929203539823, 0.815929203539823, 0.9867256637168141, 0.8972726926128776, 0.89211417513304303, 0.89211417513304303, 0.81462469147553573, 0.90132743362831846, 922.0, 208.0, 15.0, 1115.0, 1130.0, 1130.0]]]
328


True

In [42]:
"""
    cross validation
"""
classifier="KNN"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
knn = KNeighborsClassifier()
#设置k的范围
k_range = list(range(1,10))
leaf_range = list(range(1,2))
weight_options = ['uniform','distance']
algorithm_options = ['auto','ball_tree','kd_tree','brute']
param_gridknn = dict(n_neighbors = k_range,weights = weight_options,algorithm=algorithm_options,leaf_size=leaf_range)
clf = GridSearchCV(knn, param_gridknn, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_neighbors=clf.best_params_['n_neighbors']
weights=clf.best_params_['weights']
algorithm=clf.best_params_['algorithm']
leaf_size=clf.best_params_['leaf_size']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_

y_predict=cross_val_predict(KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,algorithm=algorithm,leaf_size=leaf_size),X,Y,cv=10,n_jobs=12)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,algorithm=algorithm,leaf_size=leaf_size),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_neighbors:"+str(n_neighbors)+"weights:"+str(weights)+"algorithm:"+str(algorithm)+"leaf_size:"+str(leaf_size),
            ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['KNNn_neighbors:9weights:distancealgorithm:autoleaf_size:1', 0.51725663716814163, 0.5204616998950682, 0.4389380530973451, 0.4389380530973451, 0.595575221238938, 0.5112930941090834, 0.47623619779164666, 0.47623619779164666, 0.034944623278977488, 0.51725663716814152, 496.0, 634.0, 457.0, 673.0, 1130.0, 1130.0]]]
328


True

In [43]:
"""
    cross validation XGBoost
"""
classifier="XGBoost"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
parameters = [{'n_estimators':range(1,10,1),
                      'max_depth':range(2,10),
                      'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
                      'subsample':[0.75,0.8,0.85,0.9]
                      }]
clf = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs=1, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
learning_rate=clf.best_params_['learning_rate']
subsample=clf.best_params_['subsample']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                                       subsample=subsample,max_depth=max_depth),X,Y,cv=10,n_jobs=1)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                                       subsample=subsample,max_depth=max_depth),X,Y,cv=10,n_jobs=1,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"learning_rate:"+str(learning_rate)+"subsample:"+str(subsample),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['XGBoostn_estimators:8max_depth:4learning_rate:0.5subsample:0.9', 0.84469026548672566, 0.9139213602550478, 0.7610619469026548, 0.7610619469026548, 0.9283185840707965, 0.840540272050564, 0.83051665861902457, 0.83051665861902457, 0.69923031402577851, 0.84469026548672554, 860.0, 270.0, 81.0, 1049.0, 1130.0, 1130.0]]]
328


True